In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import re

contacts = pd.read_csv('tij_LH10.dat', sep=' ', header=None, names=['timestamp', 'person_A', 'person_B'])

# Adjust the name if your metadata file is called something else (e.g., metadata_Thiers13.dat)
meta = pd.read_csv('metadata_LH10.dat', header=None, names=['raw'])

# Extract person IDs and group labels
meta['person_id'] = meta['raw'].apply(lambda x: int(re.findall(r'\d+', str(x))[0]))  # Extract number
meta['group'] = meta['raw'].apply(lambda x: re.findall(r'[A-Za-z]+', str(x))[0] if re.findall(r'[A-Za-z]+', str(x)) else None)

# Merge group labels for both persons
contacts = contacts.merge(meta[['person_id', 'group']], left_on='person_A', right_on='person_id', how='left') \
                   .rename(columns={'group': 'group_A'}).drop(columns=['person_id'])

contacts = contacts.merge(meta[['person_id', 'group']], left_on='person_B', right_on='person_id', how='left') \
                   .rename(columns={'group': 'group_B'}).drop(columns=['person_id'])

# Standardize timestamps (SocioPatterns use 20s intervals)
contacts['time_seconds'] = contacts['timestamp'] * 20
contacts['time_minutes'] = contacts['time_seconds'] / 60
contacts['time_hours'] = contacts['time_seconds'] / 3600

# Save final cleaned dataset
output_csv = "tij__merged.csv"
contacts.to_csv(output_csv, index=False)
print(f"Cleaned and merged data saved as: {output_csv}")

# Download the result
files.download(output_csv)
